In [8]:
import pandas as pd 
import numpy as np 
from collections import Counter
import xgboost
import os
import time
import warnings
import sklearn
import seaborn as sns 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt 
from sklearn.metrics import r2_score
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')

In [9]:
file=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/matrix_0.csv'
weatherfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/weather_data_2014-1-1~2016~9-30.xlsx'
holidayfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/holidays.xlsx'
if_rainfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/if_rain.xlsx'
numst='sum'

In [10]:
data=pd.read_csv(file)
weatherdata=pd.read_excel(weatherfile)
holidaydata=pd.read_excel(holidayfile,header=None)
data=data.set_index('Unnamed: 0')
k=[str(x) for x in range(1,1455)]
k.append('sum')
data.columns=k
weatherdata=weatherdata.set_index('date')
if_raindata=pd.read_excel(if_rainfile)


def tz(df,str1):
    df['timestamp']=df.index
    df['time_stamp']=pd.DataFrame(pd.to_datetime(df['timestamp']))
    df['dow'] = df.time_stamp.apply(lambda x: x.dayofweek)
    df['doy'] = df.time_stamp.apply(lambda x: x.dayofyear)
    df['day'] = df.time_stamp.apply(lambda x: x.day)
    df['is_weekend']=df.time_stamp.apply(lambda x:1 if x.dayofweek in range(5,7) else 0)
    df['month'] = df.time_stamp.apply(lambda x: x.month)
    df['d-1']=df[str1].shift(1)
    df['d-2']=df[str1].shift(2)
    df['d_dif']=df['d-1']-df['d-2']
    weatherdata['date']=weatherdata.index
    combineddf=pd.merge(left=df,right=weatherdata,left_on='time_stamp',right_on='date',how='left')
    combineddf=combineddf.fillna(method='bfill')
    return combineddf

combineddf=tz(pd.DataFrame(data[numst]),numst)
combineddf=combineddf.set_index('time_stamp')

In [16]:
df=data[data.index>='2016/07/01']

In [17]:
df.shape

(244, 1455)

In [23]:
df['40']

Unnamed: 0
2016/1/1     1
2016/1/2     1
2016/1/3     1
2016/1/4     1
2016/1/5     1
2016/1/6     1
2016/1/7     1
2016/1/8     1
2016/1/9     1
2016/1/10    1
2016/1/11    1
2016/1/12    1
2016/1/13    1
2016/1/14    1
2016/1/15    1
2016/1/16    1
2016/1/17    1
2016/1/18    1
2016/1/19    1
2016/1/20    1
2016/1/21    1
2016/1/22    1
2016/1/23    1
2016/1/24    1
2016/1/25    1
2016/1/26    1
2016/1/27    1
2016/1/28    1
2016/1/29    1
2016/1/30    1
            ..
2016/8/2     1
2016/8/3     1
2016/8/4     1
2016/8/5     1
2016/8/6     1
2016/8/7     1
2016/8/8     1
2016/8/9     1
2016/8/10    1
2016/8/11    1
2016/8/12    1
2016/8/13    1
2016/8/14    1
2016/8/15    1
2016/8/16    1
2016/8/17    1
2016/8/18    1
2016/8/19    1
2016/8/20    1
2016/8/21    1
2016/8/22    1
2016/8/23    1
2016/8/24    1
2016/8/25    1
2016/8/26    1
2016/8/27    1
2016/8/28    1
2016/8/29    1
2016/8/30    1
2016/8/31    1
Name: 40, dtype: int64

In [18]:
delete_=[]
for x in data.columns:
    if sum(df[x])==244:
        delete_.append(x)

In [24]:
delete_
for x in delete_:
    del data[x]

In [28]:
data.to_excel(r'c:\users\lance\desktop\cleaned_matrix.xlsx')